In [2]:
# Load libraries
import warnings
import numpy as np
import pandas as pd
import os
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import classification_report
SEED = 90089

In [13]:
# disable deprecation warnings and convergence warning after production 
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"

### `LogisticRegression` Classifier on Hyperclygemic patients

In [5]:
# Read data
patient_df = pd.read_csv('../data/hyperglycemic_patients_w_bin_categories.csv', usecols=list(range(3,19)))
patient_df = patient_df.drop(['los'], axis=1)

# Previewing data
print("Dataframe shape:   ", patient_df.shape)
print("Dataframe Features:", patient_df.columns)
patient_df.head()

Dataframe shape:    (12872, 15)
Dataframe Features: Index(['glucose_max', 'anchor_age', 'dod', 'gender', 'dbp_mean', 'sbp_mean',
       'glucose_mean', 'heart_rate_mean', 'spo2_mean', 'resp_rate_mean',
       'temperature_mean', 'apsiii', 'glucose_score', 'avg_bmi_value',
       'label'],
      dtype='object')


,glucose_max,anchor_age,dod,gender,dbp_mean,sbp_mean,glucose_mean,heart_rate_mean,spo2_mean,resp_rate_mean,temperature_mean,apsiii,glucose_score,avg_bmi_value,label
0,277.0,36,0,0,64.320000,112.880000,221.272727,101.800000,92.000000,17.346154,37.186250,38,3,26.750000,0
1,2340.0,35,0,0,77.769231,122.961538,468.592593,105.361111,96.093023,19.813953,36.727586,87,5,30.558477,1
2,531.0,23,1,0,64.169118,119.661765,275.352941,154.028986,83.698413,25.584615,39.725000,158,5,30.558477,2
3,259.0,19,1,1,49.300000,74.200000,259.000000,89.750000,70.000000,26.333333,31.900000,88,3,30.558477,0
4,398.0,31,0,0,78.600000,133.800000,285.000000,100.826087,90.304348,25.173913,36.593333,34,5,30.558477,0


### LR without `StandardScaler`

In [6]:
# Data split
X = patient_df.loc[:, patient_df.columns != 'label']
y = patient_df.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [20]:
# Resample data to tackle class imbalance
oversample = SMOTE()
X_sampled, y_sampled = oversample.fit_resample(X_train, y_train)

# summarize the resampled label distribution
counter = Counter(y_sampled)
print(counter)

Counter({0: 6729, 1: 6729, 2: 6729})


In [21]:
# Tune model with stratifiedKFold cross validation
lr = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [500, 1000]}
grid_lr = GridSearchCV(lr, parameters, cv=5, n_jobs=8)
grid_lr.fit(X_sampled, y_sampled)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=90089), n_jobs=8,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [500, 1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [10]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=1, max_iter=500, penalty='l1', random_state=90089,
                   solver='liblinear')

 The best parameters across ALL searched params:
 {'C': 1, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear'}


In [11]:
best_lr = LogisticRegression(C=1,
                            penalty='l1',
                            solver='liblinear',
                            random_state=SEED,
                            max_iter=500,
                            n_jobs=8)

In [22]:
best_lr.fit(X_sampled, y_sampled)

d:\miniconda3\envs\cv\lib\site-packages\sklearn\linear_model\_logistic.py:1526: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  " = {}.".format(effective_n_jobs(self.n_jobs))


LogisticRegression(C=1, max_iter=500, n_jobs=8, penalty='l1',
                   random_state=90089, solver='liblinear')

In [23]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr.score(X_test, y_test)))

Model Accuracy based on Testing dataset: 0.56777


### LR Model with `StandardScaler`

In [24]:
numerical_features = ['glucose_max', 'dbp_mean', 'sbp_mean', 'glucose_mean', 'anchor_age',
                      'heart_rate_mean', 'spo2_mean', 'resp_rate_mean', 'temperature_mean', 
                      'apsiii', 'glucose_score', 'avg_bmi_value']

# Standardized variable for consistent measurement across numerical values
scaler = StandardScaler()
X_std = scaler.fit_transform(patient_df[numerical_features])

# Interpolate categorical variables
X_cat = patient_df[['dod', 'gender']]


X = np.concatenate([X_std, X_cat.to_numpy()], axis=1)
y = patient_df["label"]
print("X Standardized data Shape: ", X_std.shape)
print("X Categorical data shape:  ", X_cat.shape)
print("X, y shape:                ", X.shape, y.shape)

X Standardized data Shape:  (12872, 12)
X Categorical data shape:   (12872, 2)
X, y shape:                 (12872, 14) (12872,)


In [25]:
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [26]:
# Resample data to tackle class imbalance
oversample = SMOTE()
X_sampled_std, y_sampled_std = oversample.fit_resample(X_train_std, y_train_std)

# summarize the resampled label distribution
counter = Counter(y_sampled_std)
print(counter)

Counter({0: 6729, 1: 6729, 2: 6729})


In [27]:
lr_std = LogisticRegression(random_state=SEED)

parameters = {
    'penalty' : ['l1', 'l2'],
    'C'       : [0.1, 1, 10],
    'solver'  : ['liblinear', 'saga'], 
    'max_iter': [100, 500, 1000]}
grid_lr_std = GridSearchCV(lr_std, parameters, cv=5, n_jobs=6)
grid_lr_std.fit(X_sampled_std, y_sampled_std)

GridSearchCV(cv=5, estimator=LogisticRegression(random_state=90089), n_jobs=6,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [100, 500, 1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'saga']})

In [28]:
print("----         Results from Grid Search           ---")
print("\n The best estimator across ALL searched params:\n", grid_lr_std.best_estimator_)
print("\n The best parameters across ALL searched params:\n", grid_lr_std.best_params_)

----         Results from Grid Search           ---

 The best estimator across ALL searched params:
 LogisticRegression(C=0.1, penalty='l1', random_state=90089, solver='saga')

 The best parameters across ALL searched params:
 {'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga'}


In [30]:
best_lr_std = LogisticRegression(C = 0.1,
                                penalty = 'l1',
                                solver = 'saga',
                                random_state=SEED,
                                max_iter=100, 
                                n_jobs=8)
best_lr_std.fit(X_sampled_std, y_sampled_std)

LogisticRegression(C=0.1, n_jobs=8, penalty='l1', random_state=90089,
                   solver='saga')

In [31]:
print("Model Accuracy based on Testing dataset: {:.5f}".format(best_lr_std.score(X_test_std, y_test_std)))

Model Accuracy based on Testing dataset: 0.56621


In [32]:
y_pred = best_lr_std.predict(X_test_std)
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.68      0.77      0.72      1462
           1       0.26      0.32      0.29       520
           2       0.58      0.28      0.38       593

    accuracy                           0.57      2575
   macro avg       0.51      0.46      0.46      2575
weighted avg       0.57      0.57      0.55      2575

